# Parquet Data Performance Testing

This notebook measures performance metrics for processing chess game data from parquet files. The main objectives are:

1. Efficiently load data from parquet files using DuckDB
2. Process the data to gather player statistics and opening preferences
3. Log detailed performance metrics (games/sec, processing time, etc.)
4. Compare with the performance of processing PGN files directly

Update: This notebook seems to do a great job of parsing games quickly. Now we are moving on to better filtering.

NOTE If you want to quickly adjust games filtering or processing parameters, ProcessingConfig is probably your best bet. I put as many relevant parameters in there as I could think of.

In [1]:
# Import necessary libraries
from typing import Dict, TypedDict, Optional, Union, Literal, Set, List
import duckdb
import time
import json
from pathlib import Path
import pandas as pd
import psutil
import pickle

## Type Definitions

Let's start by defining our data structures for strong typing support.

In [2]:
# Define types for game results
GameResult = Literal["1-0", "0-1", "1/2-1/2", "*"]

# Define types for our data structures
class OpeningResults(TypedDict):
    """Statistics for a player's results with a particular opening."""
    opening_name: str
    results: Dict[str, Union[int, float]]

class PlayerStats(TypedDict):
    """Statistics for an individual player."""
    rating: int
    white_games: Dict[str, OpeningResults]  # ECO code -> results
    black_games: Dict[str, OpeningResults]  # ECO code -> results
    num_games_total: int


class ProcessingConfig:
    """Configuration for the game processing pipeline.
    Contains parameters for filtering games, batch processing, and parallelization.
    This is designed to ensure that the processing of raw chess game data yields usable results efficiently.
    """

    def __init__(
        self,
        # Computer efficiency and organization stuff
        parquet_path: str,
        batch_size: int = 100_000,
        save_interval: int = 1,
        save_dir: str = "../data/processed",
        # Chess game filtering stuff
        # Neither the black or white player can be below this rating
        min_player_rating: int = 1200,
        # Players can't be more than 100 rating points apart
        max_elo_difference_between_players: int = 100,
        # Exclude bullet and daily games by default
        allowed_time_controls: Optional[Set[str]] = None
    ):
        # Notes on game filters:
        # Didn't exclude unrated games because our dataset contains only rated games.
        # Also didn't have to filter out bot games, because only games between two humans are rated --- I think so, at least.
        # See here to look at the data I used: https://huggingface.co/datasets/Lichess/standard-chess-games

        self.parquet_path = parquet_path # Path to the Parquet file containing raw game data
        self.batch_size = batch_size
        self.save_interval = save_interval
        self.save_dir = save_dir # Directory to save intermediate results
        self.min_player_rating = min_player_rating
        self.max_elo_difference_between_players = max_elo_difference_between_players

        # Default to common time controls if none specified
        # Exclude bullet and daily games because they're unrepresentative
        if allowed_time_controls is None:
            self.allowed_time_controls = {"Blitz", "Rapid", "Classical"}
        else:
            self.allowed_time_controls = allowed_time_controls

## Performance Metrics

We'll define a class to track performance metrics during processing.

In [3]:
class PerformanceTracker:
    """Track and report performance metrics during processing."""
    
    def __init__(self):
        self.start_time = time.time()
        self.last_log_time = self.start_time
        self.total_games = 0
        self.batch_times = []
        self.batch_sizes = []
        self.memory_usage = []
        
        # Tracking for filtered vs. accepted games
        self.accepted_games = 0
        self.filtered_games = 0
    
    def start_batch(self):
        """Mark the start of a new batch."""
        self.batch_start_time = time.time()
    
    def end_batch(self, batch_size: int):
        """Mark the end of a batch and record metrics."""
        end_time = time.time()
        batch_time = end_time - self.batch_start_time
        
        self.total_games += batch_size
        self.batch_times.append(batch_time)
        self.batch_sizes.append(batch_size)
        
        # Record memory usage
        mem = psutil.virtual_memory()
        self.memory_usage.append({
            "percent": mem.percent,
            "used_gb": mem.used / (1024**3),
            "available_gb": mem.available / (1024**3)
        })
        
        return batch_time
    
    def log_progress(self, force: bool = False):
        """Log progress information if enough time has passed or if forced."""
        current_time = time.time()
        
        # Log if it's been more than 5 seconds since the last log or if forced
        if force or (current_time - self.last_log_time) >= 5:
            elapsed_total = current_time - self.start_time
            games_per_sec = self.total_games / elapsed_total if elapsed_total > 0 else 0
            
            # Calculate recent performance (last 5 batches or fewer)
            recent_batches = min(5, len(self.batch_times))
            if recent_batches > 0:
                recent_time = sum(self.batch_times[-recent_batches:])
                recent_games = sum(self.batch_sizes[-recent_batches:])
                recent_rate = recent_games / recent_time if recent_time > 0 else 0
                
                # Get the latest memory usage
                latest_mem = self.memory_usage[-1] if self.memory_usage else {"percent": 0, "used_gb": 0, "available_gb": 0}
                
                # Calculate acceptance rate
                total_processed = self.accepted_games + self.filtered_games
                acceptance_rate = (self.accepted_games / total_processed * 100) if total_processed > 0 else 0
                
                print(f"Processed {self.total_games:,} games in {elapsed_total:.2f} seconds")
                print(f"Accepted: {self.accepted_games:,} games, Filtered: {self.filtered_games:,} games (Acceptance rate: {acceptance_rate:.1f}%)")
                print(f"Overall rate: {games_per_sec:.1f} games/sec")
                print(f"Recent rate: {recent_rate:.1f} games/sec")
                print(f"Memory usage: {latest_mem['percent']}% (Used: {latest_mem['used_gb']:.1f}GB, "
                      f"Available: {latest_mem['available_gb']:.1f}GB)")
                print("-" * 40)
            
            self.last_log_time = current_time
    
    def get_summary(self):
        """Get a summary of all performance metrics."""
        end_time = time.time()
        total_time = end_time - self.start_time
        
        avg_batch_time = sum(self.batch_times) / len(self.batch_times) if self.batch_times else 0
        max_batch_time = max(self.batch_times) if self.batch_times else 0
        min_batch_time = min(self.batch_times) if self.batch_times else 0
        
        avg_batch_size = sum(self.batch_sizes) / len(self.batch_sizes) if self.batch_sizes else 0
        
        overall_rate = self.total_games / total_time if total_time > 0 else 0
        
        # Calculate filtering stats
        total_processed = self.accepted_games + self.filtered_games
        acceptance_rate = (self.accepted_games / total_processed * 100) if total_processed > 0 else 0
        
        return {
            "total_games": self.total_games,
            "total_time_sec": total_time,
            "avg_batch_time_sec": avg_batch_time,
            "min_batch_time_sec": min_batch_time,
            "max_batch_time_sec": max_batch_time,
            "avg_batch_size": avg_batch_size,
            "overall_rate_games_per_sec": overall_rate,
            "memory_usage": self.memory_usage,
            # Add filtering stats
            "accepted_games": self.accepted_games,
            "filtered_games": self.filtered_games,
            "acceptance_rate_percent": round(acceptance_rate, 1)
        }

## Helper Functions

These functions will help us process the data and manage player statistics.

In [ ]:
def save_progress(players_data: Dict[str, PlayerStats], 
                  batch_num: int, 
                  config: ProcessingConfig,
                  perf_tracker: Optional[PerformanceTracker] = None) -> None:
    """
    Save current progress to disk atomically to prevent corruption.
    
    Args:
        players_data: Current player statistics
        batch_num: Current batch number
        config: Processing configuration
        perf_tracker: Performance tracker object
    """
    # Create save directory if it doesn't exist
    save_dir = Path(config.save_dir)
    save_dir.mkdir(parents=True, exist_ok=True)
    
    # Save player data
    player_data_path = save_dir / "player_stats_parquet.pkl"
    
    # For large datasets, pickle can be more efficient than JSON
    with open(player_data_path, 'wb') as f:
        pickle.dump(players_data, f)
        
    # Save progress information
    progress_path = save_dir / "processing_progress_parquet.json"
    
    # Create a serializable version of the config (convert set to list)
    config_dict = vars(config).copy()
    if 'allowed_time_controls' in config_dict and isinstance(config_dict['allowed_time_controls'], set):
        config_dict['allowed_time_controls'] = list(config_dict['allowed_time_controls'])
        
    progress_info = {
        "last_batch_processed": batch_num,
        "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
        "num_players": len(players_data),
        "config": config_dict
    }
    
    # Add performance metrics if available
    if perf_tracker:
        progress_info["performance"] = perf_tracker.get_summary()
    
    # Atomic write: write to a temporary file first, then rename
    temp_progress_path = progress_path.with_suffix(".json.tmp")
    try:
        with open(temp_progress_path, 'w') as f:
            json.dump(progress_info, f, indent=2)
        temp_progress_path.rename(progress_path)
    except Exception as e:
        print(f"Error saving progress: {e}")
        if temp_progress_path.exists():
            temp_progress_path.unlink()
    
    print(f"Saved progress after batch {batch_num}. " +
          f"Current data includes {len(players_data)} players.")

def load_progress(config: ProcessingConfig) -> tuple[Dict[str, PlayerStats], int]:
    """
    Load previous progress from disk, handling potential file corruption.
    
    Args:
        config: Processing configuration
        
    Returns:
        Tuple of (player_data, last_batch_processed)
    """
    player_data_path = Path(config.save_dir) / "player_stats_parquet.pkl"
    progress_path = Path(config.save_dir) / "processing_progress_parquet.json"
    
    # Default values if no saved progress
    players_data: Dict[str, PlayerStats] = {}
    last_batch = 0
    
    # Load player data if it exists
    if player_data_path.exists():
        try:
            with open(player_data_path, 'rb') as f:
                players_data = pickle.load(f)
            print(f"Loaded player data with {len(players_data)} players.")
        except Exception as e:
            print(f"Error loading player data: {e}")
            players_data = {}
    
    # Load progress info if it exists
    if progress_path.exists():
        try:
            with open(progress_path, 'r') as f:
                progress_info = json.load(f)
                last_batch = progress_info.get("last_batch_processed", 0)
            print(f"Resuming from batch {last_batch}.")
        except json.JSONDecodeError:
            print(f"Warning: Could not decode {progress_path}. File may be corrupt. Starting from scratch.")
            last_batch = 0
            players_data = {} # If progress is corrupt, start player data from scratch
        except Exception as e:
            print(f"Error loading progress info: {e}")
            last_batch = 0
            players_data = {}
            
    return players_data, last_batch

In [5]:
def is_valid_game(row: pd.Series, config: ProcessingConfig) -> bool:
    """
    Check if a game meets the filtering criteria. This ensures only relevant, informative games are processed.
    
    Args:
        row: A row from the DataFrame representing a game
        config: Processing configuration
        
    Returns:
        True if the game passes our filters, False otherwise
    """
    # Check player ratings
    if (row['WhiteElo'] < config.min_player_rating or 
        row['BlackElo'] < config.min_player_rating):
        return False

    # Check rating difference
    if abs(row['WhiteElo'] - row['BlackElo']) > config.max_elo_difference_between_players:
        return False

    # "Event" column on game contains time control, they're titled like "Rated Blitz Games"
    # Check that the time control is in the allowed time controls (case insensitive)
    event_lower = row["Event"].lower()
    if not any(tc.lower() in event_lower for tc in config.allowed_time_controls):
        return False

    # Check for valid result
    # If it's something weird that's not a win loss or draw, toss it out
    if row['Result'] not in {"1-0", "0-1", "1/2-1/2"}:
        return False

    return True

## Data Processing Functions

Now let's implement the core functions to process the parquet data efficiently.

In [ ]:
def process_batch(batch_df: pd.DataFrame, 
                  players_data: Dict[str, PlayerStats],
                  config: ProcessingConfig, 
                  log_frequency: int = 5000, 
                  perf_tracker: Optional[PerformanceTracker] = None, 
                  file_context: Optional[Dict] = None) -> None:
    """
    Process a batch of games and update the main players_data dictionary directly.
    This function iterates through each game in a batch, filters it, and then
    updates the statistics for the white and black players in the provided
    players_data dictionary. This in-place update strategy avoids the need
    for merging separate dictionaries later, preventing data duplication bugs.
    
    Args:
        batch_df: DataFrame containing a batch of games.
        players_data: The main dictionary of player statistics to be updated.
        config: Processing configuration.
        log_frequency: Log progress after processing this many games.
        perf_tracker: Performance tracker object to update with filtering stats.
        file_context: Dictionary with context about the multi-file processing job.
    """
    start_time = time.time()
    total_rows = len(batch_df)
    
    # Tracking for filtered vs. accepted games in this batch
    batch_accepted = 0
    batch_filtered = 0
    
    # Process each game in the batch
    for i, (_, game) in enumerate(batch_df.iterrows()):
        # Log progress periodically within the batch
        if (i + 1) % log_frequency == 0:
            elapsed = time.time() - start_time
            rate = (i + 1) / elapsed if elapsed > 0 else 0
            eta = (total_rows - (i + 1)) / rate if rate > 0 else 0
            
            # Calculate acceptance rate for this batch so far
            processed_so_far = batch_accepted + batch_filtered
            acceptance_rate = (batch_accepted / processed_so_far * 100) if processed_so_far > 0 else 0
            
            # Multi-file progress
            multi_file_str = ""
            if file_context:
                files_remaining = file_context['total_files'] - file_context['current_file_num']
                
                # Estimate total progress
                rows_done_in_prev_files = (file_context['current_file_num'] - 1) * file_context['avg_rows_per_file']
                rows_done_in_current_file = i + 1
                total_rows_processed = rows_done_in_prev_files + rows_done_in_current_file
                
                total_elapsed = time.time() - file_context['total_start_time']
                overall_rate = total_rows_processed / total_elapsed if total_elapsed > 0 else 0
                
                remaining_rows = file_context['total_rows_estimate'] - total_rows_processed
                total_eta_seconds = remaining_rows / overall_rate if overall_rate > 0 else 0
                
                multi_file_str = (
                    f"File {file_context['current_file_num']}/{file_context['total_files']} "
                    f"({files_remaining} left) - Total ETA: {total_eta_seconds / 60:.1f} min"
                )

            print(f"Progress: {i+1:,}/{total_rows:,} ({(i+1)/total_rows*100:.1f}%) - "
                  f"Rate: {rate:.1f} games/sec - File ETA: {eta/60:.1f} min - {multi_file_str}")
            print(f"Batch filtering: Accepted {batch_accepted:,}, Filtered {batch_filtered:,} (Acceptance rate: {acceptance_rate:.1f}%)")
        
        # Filter out invalid games
        if not is_valid_game(game, config):
            batch_filtered += 1
            if perf_tracker:
                perf_tracker.filtered_games += 1
            continue
        
        # Mark as accepted
        batch_accepted += 1
        if perf_tracker:
            perf_tracker.accepted_games += 1

        # Extract relevant fields
        white_player = game['White']
        black_player = game['Black']
        
        # Handle potential missing values
        try:
            white_elo = int(game.get('WhiteElo', 0))
            black_elo = int(game.get('BlackElo', 0))
        except (ValueError, TypeError):
            white_elo = 0
            black_elo = 0
            
        result = game['Result']
        eco_code = game.get('ECO', 'Unknown')
        opening_name = game.get('Opening', 'Unknown Opening')
        
        # Process white player's game
        if white_player not in players_data:
            players_data[white_player] = {
                "rating": white_elo,
                "white_games": {},
                "black_games": {},
                "num_games_total": 0
            }
        
        # Update white player's data
        if eco_code not in players_data[white_player]["white_games"]:
            players_data[white_player]["white_games"][eco_code] = {
                "opening_name": opening_name,
                "results": {
                    "num_games": 0,
                    "num_wins": 0,
                    "num_losses": 0,
                    "num_draws": 0,
                    "score_percentage_with_opening": 0
                }
            }
        
        # Update game counts
        players_data[white_player]["num_games_total"] += 1
        players_data[white_player]["white_games"][eco_code]["results"]["num_games"] += 1
        
        # Update result counts
        if result == "1-0":  # White win
            players_data[white_player]["white_games"][eco_code]["results"]["num_wins"] += 1
        elif result == "0-1":  # Black win (white loss)
            players_data[white_player]["white_games"][eco_code]["results"]["num_losses"] += 1
        elif result == "1/2-1/2":  # Draw
            players_data[white_player]["white_games"][eco_code]["results"]["num_draws"] += 1
            
        # Update score percentage
        wins = players_data[white_player]["white_games"][eco_code]["results"]["num_wins"]
        draws = players_data[white_player]["white_games"][eco_code]["results"]["num_draws"]
        total = players_data[white_player]["white_games"][eco_code]["results"]["num_games"]
        score = (wins + (draws * 0.5)) / total * 100 if total > 0 else 0
        players_data[white_player]["white_games"][eco_code]["results"]["score_percentage_with_opening"] = round(score, 1)
        
        # Similarly process black player's game
        if black_player not in players_data:
            players_data[black_player] = {
                "rating": black_elo,
                "white_games": {},
                "black_games": {},
                "num_games_total": 0
            }
        
        # Update black player's data
        if eco_code not in players_data[black_player]["black_games"]:
            players_data[black_player]["black_games"][eco_code] = {
                "opening_name": opening_name,
                "results": {
                    "num_games": 0,
                    "num_wins": 0,
                    "num_losses": 0,
                    "num_draws": 0,
                    "score_percentage_with_opening": 0
                }
            }
        
        # Update game counts
        players_data[black_player]["num_games_total"] += 1
        players_data[black_player]["black_games"][eco_code]["results"]["num_games"] += 1
        
        # Update result counts
        if result == "0-1":  # Black win
            players_data[black_player]["black_games"][eco_code]["results"]["num_wins"] += 1
        elif result == "1-0":  # White win (black loss)
            players_data[black_player]["black_games"][eco_code]["results"]["num_losses"] += 1
        elif result == "1/2-1/2":  # Draw
            players_data[black_player]["black_games"][eco_code]["results"]["num_draws"] += 1
            
        # Update score percentage
        wins = players_data[black_player]["black_games"][eco_code]["results"]["num_wins"]
        draws = players_data[black_player]["black_games"][eco_code]["results"]["num_draws"]
        total = players_data[black_player]["black_games"][eco_code]["results"]["num_games"]
        score = (wins + (draws * 0.5)) / total * 100 if total > 0 else 0
        players_data[black_player]["black_games"][eco_code]["results"]["score_percentage_with_opening"] = round(score, 1)
    
    # Final progress update
    elapsed = time.time() - start_time
    rate = total_rows / elapsed if elapsed > 0 else 0
    acceptance_rate = (batch_accepted / total_rows * 100) if total_rows > 0 else 0
    print(f"Completed {total_rows:,} games in {elapsed:.1f} seconds - Rate: {rate:.1f} games/sec")
    print(f"Batch filtering stats: Accepted {batch_accepted:,}, Filtered {batch_filtered:,} (Acceptance rate: {acceptance_rate:.1f}%)")
    

## Main Processing Function

Now let's implement the main function that processes the parquet file in batches.

## Multi-File Processing with Duplicate Detection

Before defining our main processing function, let's implement a system to handle duplicate file detection. This is essential when processing multiple parquet files that might have similar names but come from different months or batches. Our approach uses metadata fingerprinting to uniquely identify each file.

This uses the dupe-check utils defined in our utils folder.

In [7]:
# Import our custom file registry utility
import sys
from pathlib import Path

# Add the parent directory to the Python path to enable imports
notebooks_dir = Path.cwd().parent
if str(notebooks_dir) not in sys.path:
    sys.path.append(str(notebooks_dir))

# Try to import our custom file registry utility
try:
    from notebooks.utils.file_processing.raw_data_file_dupe_checks import FileRegistry
    print("Successfully imported FileRegistry")
except ImportError:
    print("Could not import FileRegistry - file duplicate checks will not be available")
    
    # Define a simple FileRegistry class if the import fails
    class FileRegistry:
        """Simple FileRegistry implementation for duplicate detection."""
        def __init__(self):
            self.registry_path = Path(notebooks_dir) / "data/processed/file_registry.json"
            self.processed_files = set()
            self._load_registry()
            
        def _load_registry(self):
            import json
            if self.registry_path.exists():
                try:
                    with open(self.registry_path, 'r') as f:
                        data = json.load(f)
                        self.processed_files = set(data.get('processed_files', []))
                except Exception as e:
                    print(f"Warning: Could not load registry: {e}")
        
        def _save_registry(self):
            import json
            try:
                self.registry_path.parent.mkdir(parents=True, exist_ok=True)
                with open(self.registry_path, 'w') as f:
                    json.dump({'processed_files': list(self.processed_files)}, f)
            except Exception as e:
                print(f"Warning: Could not save registry: {e}")
                
        def is_file_processed(self, file_path: str) -> bool:
            return str(file_path) in self.processed_files
            
        def mark_file_processed(self, file_path: str) -> None:
            self.processed_files.add(str(file_path))
            self._save_registry()
            
        def mark_file_skipped(self, file_path: str) -> None:
            self.processed_files.add(str(file_path))
            self._save_registry()

Successfully imported FileRegistry


## System Information

Let's check the system's hardware resources to optimize our configuration.

In [ ]:
# Install psutil if not already installed
import sys
import subprocess

try:
    import psutil
except ImportError:
    print("Installing psutil package...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "psutil"])
    import psutil
    print("psutil installed successfully")

def get_system_info():
    """Get information about the system's hardware resources."""
    info = {
        "cpu_count_physical": psutil.cpu_count(logical=False),
        "cpu_count_logical": psutil.cpu_count(logical=True),
        "memory_total_gb": round(psutil.virtual_memory().total / (1024**3), 2),
        "memory_available_gb": round(psutil.virtual_memory().available / (1024**3), 2)
    }
    return info

# Get system information
system_info = get_system_info()
print("System Information:")
for key, value in system_info.items():
    print(f"  {key}: {value}")

# Calculate optimal batch size based on available memory
# Assuming each row needs about 1KB of memory
available_memory_gb = system_info["memory_available_gb"]
memory_for_batch_gb = available_memory_gb * 0.3  # Use 30% of available memory
optimal_batch_size = int(memory_for_batch_gb * 1024**3 / 1024)  # 1KB per row

# Round to nearest 10,000
optimal_batch_size = max(10_000, round(optimal_batch_size / 10_000) * 10_000)

print(f"\nRecommended batch size based on memory: {optimal_batch_size:,}")

def process_parquet_file(config: ProcessingConfig, 
                         players_data: Dict[str, PlayerStats],
                         log_frequency: int = 5000, 
                         file_context: Optional[Dict] = None) -> None:
    """
    Process a single parquet file in batches, updating a shared players_data dictionary.
    This function orchestrates the processing of one file. It uses a file registry
    to skip already processed files and manages batching. For each batch, it calls
    process_batch to perform the actual game data processing.
    
    Args:
        config: Processing configuration for this specific file.
        players_data: The shared dictionary of player statistics to update.
        log_frequency: How often to log progress within a batch.
        file_context: Dictionary with context for multi-file processing.
    """
    # Check if file has already been processed using FileRegistry
    try:
        registry = FileRegistry()
        if registry.is_file_processed(config.parquet_path):
            print(f"Skipping already processed file: {Path(config.parquet_path).name}")
            return
    except Exception as e:
        print(f"Warning: Could not check file registry: {e}")
    
    # Initialize DuckDB connection
    con = duckdb.connect()
    
    # For a single file run, we might need to load progress.
    # In a multi-file run, players_data is already populated.
    if not players_data:
        loaded_data, start_batch = load_progress(config)
        # This is a bit of a hack to get the loaded data out
        for k, v in loaded_data.items():
            players_data[k] = v
    else:
        start_batch = 0

    # Initialize performance tracker
    perf_tracker = PerformanceTracker()
    
    # Get total number of rows
    print("Counting total rows in parquet file...")
    total_rows = con.execute(
        f"SELECT COUNT(*) FROM '{config.parquet_path}'"
    ).fetchone()[0]
    print(f"Total rows in parquet file: {total_rows:,}")
    
    # If resuming, check if we're already done
    if start_batch * config.batch_size >= total_rows:
        print(f"Resuming from batch {start_batch}, which is after the end of the file. Skipping.")
        # Mark the file as processed since we are technically done with it
        try:
            registry.mark_file_processed(config.parquet_path)
            print(f"Marked file {Path(config.parquet_path).name} as processed in the registry")
        except Exception as e:
            print(f"Warning: Could not update file registry: {e}")
        return players_data

    # Calculate number of batches
    total_batches = (total_rows + config.batch_size - 1) // config.batch_size
    print(f"Will process in {total_batches} batches of size {config.batch_size:,} (resuming from batch {start_batch})")
    
    # Process in batches
    batch_num = start_batch
    
    while True:
        # Calculate offset for the current batch
        offset = batch_num * config.batch_size
        
        # Check if we've processed all rows
        if offset >= total_rows:
            print("Processed all rows. Finishing up.")
            break
        
        print(f"\nProcessing batch {batch_num + 1}/{total_batches} (offset {offset:,})")
        perf_tracker.start_batch()
        
        # Fetch a batch of data
        batch_query = f"""
        SELECT 
            Event, White, Black, Result, 
            WhiteTitle, BlackTitle, WhiteElo, BlackElo, 
            WhiteRatingDiff, BlackRatingDiff, ECO, Opening,
            Termination, TimeControl
        FROM '{config.parquet_path}'
        LIMIT {config.batch_size} OFFSET {offset}
        """
        
        # Execute the query and convert to DataFrame
        batch_df = con.execute(batch_query).df()
        
        if batch_df.empty:
            print("Loaded an empty batch. This might mean we are past the end of the file.")
            break

        print(f"Loaded batch with {len(batch_df):,} rows")
        
        # Process the batch, updating players_data in-place
        process_batch(
            batch_df, 
            players_data, 
            config, 
            log_frequency=log_frequency, 
            perf_tracker=perf_tracker, 
            file_context=file_context
        )
        
        # Record batch completion
        batch_time = perf_tracker.end_batch(len(batch_df))
        print(f"Processed batch in {batch_time:.2f} seconds")
        print(f"Current player count: {len(players_data):,}")
        
        # Log performance metrics
        perf_tracker.log_progress(force=True)
        
        # Save progress periodically
        batch_num += 1
        if batch_num % config.save_interval == 0:
            save_progress(players_data, batch_num, config, perf_tracker)
    
    # Save final progress
    save_progress(players_data, batch_num, config, perf_tracker)
    
    # Mark the file as processed in the registry
    try:
        registry.mark_file_processed(config.parquet_path)
        print(f"Marked file {Path(config.parquet_path).name} as processed in the registry")
    except Exception as e:
        print(f"Warning: Could not update file registry: {e}")
    
    # Print final performance summary
    summary = perf_tracker.get_summary()
    print("\nPerformance Summary:")
    print(f"Total games processed: {summary['total_games']:,}")
    print(f"Total processing time: {summary['total_time_sec']:.2f} seconds")
    print(f"Overall processing rate: {summary['overall_rate_games_per_sec']:.2f} games/second")
    print(f"Average batch processing time: {summary['avg_batch_time_sec']:.2f} seconds")
    
    # Add filtering stats to the final summary
    print("\nFiltering Statistics:")
    print(f"Accepted games: {summary['accepted_games']:,}")
    print(f"Filtered out games: {summary['filtered_games']:,}")
    print(f"Acceptance rate: {summary['acceptance_rate_percent']}%")
    
    return players_data

System Information:
  cpu_count_physical: 6
  cpu_count_logical: 12
  memory_total_gb: 32.0
  memory_available_gb: 23.55

Recommended batch size based on memory: 7,410,000


In [ ]:
def process_multiple_parquet_files(file_paths: List[str], base_config: ProcessingConfig = None, log_frequency: int = 5000) -> Dict[str, PlayerStats]:
    """
    Process multiple parquet files of raw game data, avoiding duplicates.
    This is the main entry point for processing a collection of files. It initializes
    a single 'all_players_data' dictionary that is shared and updated across all
    file processing jobs. It handles duplicate file detection and orchestrates
    the processing of each file by calling 'process_parquet_file'.
    
    Args:
        file_paths: List of paths to parquet files to process.
        base_config: Base configuration to use as a template for each file.
        log_frequency: How often to log progress within a batch.
        
    Returns:
        The final, combined player statistics from all processed files.
    """
    if not file_paths:
        print("No files provided for processing")
        return {}
    
    # Create a default config if none provided
    if base_config is None:
        base_config = ProcessingConfig(
            parquet_path="",  # Will be set for each file
            batch_size=100_000,
            save_interval=1,
            save_dir="../data/processed"
        )
    
    # Initialize file registry
    try:
        registry = FileRegistry()
    except NameError:
        print("FileRegistry not available, skipping duplicate detection")
        registry = None
    
    # Filter out already processed files
    new_files = []
    for file_path in file_paths:
        if registry and registry.is_file_processed(file_path):
            print(f"Skipping already processed file: {Path(file_path).name}")
            try:
                registry.mark_file_skipped(file_path)
            except Exception as e:
                print(f"Warning: Could not mark file as skipped: {e}")
            continue
        new_files.append(file_path)
    
    if not new_files:
        print("No new files to process.")
        return {}
    
    print(f"Found {len(new_files)} new files to process out of {len(file_paths)} total files.")
    
    # Estimate total rows for ETA calculation
    # For simplicity, we'll get the row count of the first file and multiply
    total_rows_estimate = 0
    avg_rows_per_file = 0
    if new_files:
        try:
            con = duckdb.connect()
            first_file_rows = con.execute(f"SELECT COUNT(*) FROM '{new_files[0]}'").fetchone()[0]
            avg_rows_per_file = first_file_rows
            total_rows_estimate = avg_rows_per_file * len(new_files)
            con.close()
            print(f"Estimating total of {total_rows_estimate:,} rows across {len(new_files)} files for ETA.")
        except Exception as e:
            print(f"Warning: Could not estimate total rows for ETA: {e}")

    # Load all existing player data and progress once at the beginning.
    # This single dictionary will be passed to and updated by all subsequent function calls.
    all_players_data, _ = load_progress(base_config)
    total_start_time = time.time()
    
    for i, file_path in enumerate(new_files):
        print(f"\nProcessing file {i+1}/{len(new_files)}: {Path(file_path).name}")
        
        # Create a config for this specific file
        file_config = ProcessingConfig(
            parquet_path=file_path,
            batch_size=base_config.batch_size,
            save_interval=base_config.save_interval,
            save_dir=base_config.save_dir,
            min_player_rating=base_config.min_player_rating,
            max_elo_difference_between_players=base_config.max_elo_difference_between_players,
            allowed_time_controls=base_config.allowed_time_controls
        )
        
        file_context = {
            "current_file_num": i + 1,
            "total_files": len(new_files),
            "total_rows_estimate": total_rows_estimate,
            "avg_rows_per_file": avg_rows_per_file,
            "total_start_time": total_start_time,
        }

        try:
            # Process the file, passing in the single, shared data dictionary
            # to be updated in-place.
            process_parquet_file(
                file_config, 
                all_players_data,
                log_frequency=log_frequency,
                file_context=file_context
            )
            
        except Exception as e:
            print(f"Error processing {file_path}: {str(e)}")
    
    return all_players_data

## Run Processing

Now let's run the processing with our multi-file processing utility. This will allow us to process multiple parquet files at once while handling duplicate detection.

In [10]:
# Import our multi-file processing utility
from notebooks.utils.file_processing.process_multiple_raw_files import process_multiple_files

# Get the processing configuration using the utility
# This will show a directory picker dialog and find all parquet files
processing_config = process_multiple_files(
    # Let the user select a directory via dialog
    directory=None,  
    # Determine batch size automatically based on memory
    batch_size=None,  
    # Use the same filtering parameters as before
    min_player_rating=1200,
    max_elo_difference=100,
    allowed_time_controls={"Blitz", "Rapid", "Classical"},
    save_dir="../data/processed"
)

print("\nProcessing Configuration:")
for key, value in processing_config.items():
    if key != "files_to_process":  # Don't print the full file paths
        print(f"  {key}: {value}")
    else:
        print(f"  {key}: {len(value)} files")
        # Print first 3 file names as examples
        for i, file_path in enumerate(value[:3]):
            print(f"    - {Path(file_path).name}")
        if len(value) > 3:
            print(f"    - ... and {len(value) - 3} more files")

files_to_process = processing_config.get("files_to_process", [])

if not files_to_process:
    print("No new files to process. Exiting.")
else:
    print(f"Starting processing of {len(files_to_process)} files...")
    
    base_config = ProcessingConfig(
        parquet_path="",  # Will be set for each file
        batch_size=processing_config["batch_size"],
        save_interval=1,  # Save after each batch
        save_dir=processing_config["save_dir"],
        min_player_rating=processing_config["min_player_rating"],
        max_elo_difference_between_players=processing_config["max_elo_difference"],
        allowed_time_controls=processing_config["allowed_time_controls"]
    )
    
    all_players_data = process_multiple_parquet_files(
        files_to_process,
        base_config=base_config,
        log_frequency=5000
    )
    
    print("\nFinal combined data statistics:")
    print(f"Total number of players: {len(all_players_data):,}")
    
    # Save the final merged data separately
    final_save_path = Path(processing_config["save_dir"]) / "all_players_stats_combined.pkl"
    with open(final_save_path, 'wb') as f:
        pickle.dump(all_players_data, f)
    
    print(f"Saved final merged data to: {final_save_path}")
    
    # Show an example player from the combined data
    if all_players_data:
        import random
        sample_player = random.choice(list(all_players_data.keys()))
        print(f"\nSample stats for player from combined data: {sample_player}")
        print(f"Rating: {all_players_data[sample_player]['rating']}")
        print(f"Total games: {all_players_data[sample_player]['num_games_total']}")
        
        print("\nTop White openings:")
        white_openings = all_players_data[sample_player]['white_games']
        if white_openings:
            # Sort by number of games
            sorted_openings = sorted(
                white_openings.items(), 
                key=lambda x: x[1]['results']['num_games'], 
                reverse=True
            )
            for eco, data in sorted_openings[:5]: 
                print(f"  {eco} - {data['opening_name']}: " +
                      f"{data['results']['score_percentage_with_opening']}% score in {data['results']['num_games']} games")
            if len(white_openings) > 5:
                print(f"  ... and {len(white_openings) - 5} more openings")
        else:
            print("  No white openings")

        print("\nTop Black openings:")
        black_openings = all_players_data[sample_player]['black_games']
        if black_openings:
            # Sort by number of games
            sorted_openings = sorted(
                black_openings.items(), 
                key=lambda x: x[1]['results']['num_games'], 
                reverse=True
            )
            for eco, data in sorted_openings[:5]: 
                print(f"  {eco} - {data['opening_name']}: " + 
                      f"{data['results']['score_percentage_with_opening']}% score in {data['results']['num_games']} games")
            if len(black_openings) > 5:
                print(f"  ... and {len(black_openings) - 5} more openings")
        else:
            print("  No black openings")

Successfully imported FileRegistry
Found 22 parquet files in /Users/a/Documents/personalprojects/chess-opening-recommender/data/raw
Loaded registry with 12 processed files
Skipping already processed file: train-00005-of-00066.parquet
Skipping already processed file: train-00001-of-00066.parquet
Skipping already processed file: train-00010-of-00066.parquet
Skipping already processed file: train-00004-of-00066.parquet
Skipping already processed file: train-00000-of-00066.parquet
Skipping already processed file: train-00007-of-00066.parquet
Skipping already processed file: train-00008-of-00066.parquet
Skipping already processed file: train-00000-of-00072.parquet
Skipping already processed file: train-00003-of-00066.parquet
Skipping already processed file: train-00006-of-00066.parquet
Skipping already processed file: train-00009-of-00066.parquet
Skipping already processed file: train-00002-of-00066.parquet
Will process 10 new files out of 22 total files.
Using automatically determined batc

## Usage Instructions

To process multiple parquet files:

1. Run the cell above that calls `process_multiple_files()`
2. A directory picker dialog will appear - select the folder containing your parquet files
3. The utility will identify new files (not previously processed) and process them one by one
4. All player statistics will be merged into a combined dataset

You can keep adding new parquet files to the same directory, and when you run this notebook again it will only process the new ones. This is perfect for incrementally adding to your dataset over time.